# 自動微分

In [1]:
%matplotlib inline


In [2]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)


モデルの重みなどのパラメータは損失関数に対する変数の勾配（微分値）に応じて調整される。
この勾配の値を計算するために`torch.autograd`という微分エンジンが組み込まれている。

In [3]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)


Gradient function for z = <AddBackward0 object at 0x7ff9f0fb3a00>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7ff9f0fb3850>


In [4]:
loss.backward()
print(w.grad)
print(b.grad)


tensor([[0.0624, 0.1311, 0.0157],
        [0.0624, 0.1311, 0.0157],
        [0.0624, 0.1311, 0.0157],
        [0.0624, 0.1311, 0.0157],
        [0.0624, 0.1311, 0.0157]])
tensor([0.0624, 0.1311, 0.0157])


NNの各パラメータを最適化するために、損失関数の各変数の偏微分値を計算する必要がある。
これらの偏微分地を求めるために`loss.backward()`を実行し、`w.grad`と`b.grad`に勾配を格納する。

`requires_grad=True`であるすべてのテンソルは計算履歴が保持され、勾配計算可能な状態だが、勾配計算不要な場合（訓練済みモデルで推論するケースなど）は、`torch.no_grad()`で囲むことで勾配計算を無効化できる。

In [5]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)


True
False


In [6]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)


False


勾配の計算、追跡を不能にしたいケース：
- モデルのパラメータを固定したい場合
- 順伝搬の計算スピードを高速化したい場合